In [1]:
import json
import pandas as pd
from equipartition import Equipartition

# First, read in the files with the literature best-fit parameters

In [2]:
lit = pd.read_csv("lit_sed_params.csv", comment="#")
bhmasses = pd.read_csv("bh_masses_host.txt", sep="\t")

lit

,Name,t,p,epsilon_e,epsilon_B,nu_a,nu_a_error,nu_m,nu_m_error,Fp_mJy,Fp_mJy_error,dist_Mpc,z
0,CNSSJ0019+00,573,3.3,0.1,0.100,4.260000,0.040000,-99.990000,-99.990000,8.270000,0.030,77.1,0.0180
1,CNSSJ0019+00,606,3.3,0.1,0.100,3.870000,0.030000,-99.990000,-99.990000,8.080000,0.040,77.1,0.0180
2,CNSSJ0019+00,731,3.3,0.1,0.100,3.120000,0.040000,-99.990000,-99.990000,7.410000,0.060,77.1,0.0180
3,CNSSJ0019+00,998,3.3,0.1,0.100,1.920000,0.060000,-99.990000,-99.990000,4.950000,0.090,77.1,0.0180
4,ASASSN-14li,128,3.0,0.1,0.100,16.800000,0.000000,-99.990000,-99.990000,1.910000,0.000,90.0,0.0206
5,ASASSN-14li,143,3.0,0.1,0.100,8.200000,0.100000,-99.990000,-99.990000,1.760000,0.010,90.0,0.0206
6,ASASSN-14li,207,3.0,0.1,0.100,4.370000,0.200000,-99.990000,-99.990000,1.230000,0.030,90.0,0.0206
7,ASASSN-14li,246,3.0,0.1,0.100,4.000000,0.060000,-99.990000,-99.990000,1.140000,0.010,90.0,0.0206
8,ASASSN-14li,304,3.0,0.1,0.100,2.550000,0.060000,-99.990000,-99.990000,0.940000,0.020,90.0,0.0206
9,ASASSN-14li,381,3.0,0.1,0.100,1.910000,0.070000,-99.990000,-99.990000,0.620000,0.020,90.0,0.0206


# Now, use the Equipartition class to compute physical properties of the system

In [3]:
help(Equipartition)

Help on class Equipartition in module equipartition.equipartition:

class Equipartition(builtins.object)
 |  Equipartition(FpmJy, nup10, tdays, z, theta, R17=None, nuM10=1, nuA10=1, fA=1, fV=1, fOmega=1, p=2, onAxis=True, newtonian=False, table=False, tol=1e-15, maxiter=500, epse=0.1, epsB=None, corr=True, BDfactor=False, gammaM_newtonian=2, hotprotons=True, numelectrons=True, outofequipartition=True, isoNewtonianNe=False, cosmo=FlatLambdaCDM(name='Planck18', H0=<Quantity 67.66 km / (Mpc s)>, Om0=0.30966, Tcmb0=<Quantity 2.7255 K>, Neff=3.046, m_nu=<Quantity [0.  , 0.  , 0.06] eV>, Ob0=0.04897), factorsFour=False)
 |
 |  Equipartition calculator based on Matsumoto and Piran 2023 (MP23)
 |  prescription (https://doi.org/10.1093/mnras/stad1269). cgs + Gauss units
 |
 |  Methods defined here:
 |
 |  C(self)
 |      Shen and Zhang (A15) (https://doi.org/10.1111/j.1365-2966.2009.15212.x).
 |      MP23 and Barniol Duran et al. 2013 (https://doi.org/10.1088/0004-637X/772/1/78)
 |      take th